In [12]:
from instagram_web_api import Client, ClientCompatPatch, ClientError, ClientLoginError
from datetime import datetime

import hashlib
import string
import random
import time
import json
import os.path
import logging
import argparse

# просто Client не работает, поэтому переопределена функция _extract_rhx_gis для захода в web инста
class MyClient(Client):
    @staticmethod
    def _extract_rhx_gis(html):
        options = string.ascii_lowercase + string.digits
        text = ''.join([random.choice(options) for _ in range(8)])
        return hashlib.md5(text.encode())

def getWebAPI(username, password):
    return MyClient(auto_patch = True, authenticate = True, username = username, password = password)

In [13]:
class Comment:
    
    # просто структурка для коммента, в которой хранятся автор, время и текст коммента
    
    def __init__(self, com_author, com_time, com_text):
        self.com_author = com_author
        self.com_time = com_time
        self.com_text = com_text
    
class Post:
    
    # структурка для поста: id поста и число комментов к нему
    
    def __init__(self, media_id, comments_count):
        self.media_id = media_id
        self.comments_count = comments_count
    
# возвращает id юзера по нику
def getUserID(api, username):
    return api.user_info2(username)['id']

# возвращает media_id всех медиа профиля
def getProfileMediaList(api, username):
    user_id = getUserID(api, username)
    
    media_list = []
    next_page = ''
    has_next_page = True
    
    while (has_next_page):
        
        time.sleep(3) # ждем некоторое время, так как нельзя подряд делать бесконечное число http-запросов
        
        all_media = api.user_feed(user_id, count = 50, end_cursor = next_page, extract = False)
        
        # записываем id полученных медиа профиля в лист
        for item in all_media['data']['user']['edge_owner_to_timeline_media']['edges']:
            media_list.append(Post(item['node']['shortcode'], item['node']['edge_media_preview_comment']['count']))
        
        # смотрим, есть ли следующая страница
        has_next_page = all_media['data']['user']['edge_owner_to_timeline_media']['page_info']['has_next_page']
        
        # присваиваем ссылку на следующую страницу
        next_page = all_media['data']['user']['edge_owner_to_timeline_media']['page_info']['end_cursor']
        
    return media_list

# возвращает список всех комментов к посту
def getPostComments(api, media_id):
    res = []
    max_id = ' '
    next_max_id = ''
    
    while (max_id != next_max_id):
        
        time.sleep(3) # ждем некоторое время, так как нельзя подряд делать бесконечное число http-запросов
        
        max_id = next_max_id
        post_comments = api.media_comments(media_id, count = 50, end_cursor = max_id) # загружаем комменты
        
        # если комменты кончились, ливаем
        if (len(post_comments) == 0):
            break
        
        # смотрим, на каком комменте закончилась выгрузка
        # реверсим массив комментов, чтобы итоговый список получился в убывающем порядке
        next_max_id = post_comments[0]['id']
        post_comments.reverse()
        
        # оставляем только нужную информацию
        for comm in post_comments:
            res.append(Comment(comm['owner']['username'], comm['created_at'], comm['text']))
    
    return res

# возвращает все комменты к списку постов
def getPostsComments(api, posts):
    res = []
    for item in posts:
        res.extend(getPostComments(api, item.media_id))
    
    return res

# возвращает общее число комментариев к списку постов
def getSumCommentsCount(posts):
    cnt = 0
    for item in posts:
        cnt += item.comments_count

    return cnt

# возвращает общее число постов с комментариями из заданного списка постов
def getPostsWithCommentsCount(posts):
    cnt = 0
    for item in posts:
        if (item.comments_count > 0):
            cnt += 1

    return cnt

In [14]:
def getProfileStatistics(api, username):
    # получаем список постов в профиле
    posts = getProfileMediaList(api, username)
    
    # общее число постов пользователя
    posts_cnt = len(posts)
    print('posts count: ' + str(posts_cnt))
    
    # общее число постов с комментами
    posts_with_comments_cnt = getPostsWithCommentsCount(posts)
    print('posts with comments count: ' + str(posts_with_comments_cnt))
    
    # общее число постов без комментов
    posts_without_comments_cnt = posts_cnt - posts_with_comments_cnt
    print('posts without comments count: ' + str(posts_without_comments_cnt))
    
    # общее число комментов к профилю
    comments_cnt = getSumCommentsCount(posts)
    print('comments count: ' + str(comments_cnt))
    
    # получаем список всех комментов в профиле
    comments = getPostsComments(api, posts)
    
    for item in comments:
        print(item.com_text)

In [15]:
api = getWebAPI('testinstapp', 'testinstapp1')
username = 'mr_justadog' # colesprouse mr_justadog

getProfileStatistics(api, username)

posts count: 44
posts with comments count: 37
posts without comments count: 7
comments count: 138
@holymosh ты просто первую полностью не видел)0
А тебе на второй фотке ещё налили?)
@permyakovaart не выдержала психологического давления
Алло, сегодня опять блистаю🤩
А где мотимотичька?
@erzhenavamp еее
подпись кайф
@permyakovaart да говорю же, мотемотичька зверствует
Да на что там опять все смотрят?!
Офигенный костюм, мне очень зашёл)
@anasta_su ахуена
@permyakovaart ЧЕТВЕРтовали.
@mr_justadog вы ее повесили?!?
@permyakovaart мотемотичька опять довела
Гспд что там? Мне теперь интересно
@09.dzhamitova спасибо)
Дорогой племянник, с днем рождения!!!! Всех благ!!!
@egd09111981 🙌🙌
Звезда эстрады👍🔥
@nekatyan обидно
@mr_justadog ты так выглядишь
@nekatyan почему из русского((
Ты как будто из русского сериала вылез
@mr_justadog из-за твоего пения в Шлаковке 3ю школу закрыли :D
@permyakovaart поздно
Nooo
🙉
@mypaka1 а спать как ?
Найдите себе такого человека, который будет смотреть на вас так же
#